<a href="https://colab.research.google.com/github/EshwarDhande/Model-Tweaks/blob/main/Quickplay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import torch

In [1]:

!pip install transformers

from transformers import AutoModel, AutoTokenizer

model_name = 'bert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [3]:
print(model)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [4]:
seventh_layer = model.encoder.layer[6]

In [5]:
attention_weights = seventh_layer.attention.self.query.weight
attention_biases = seventh_layer.attention.self.query.bias

print("Original attention weights shape:", attention_weights.shape)
print("Original attention biases shape:", attention_biases.shape)


Original attention weights shape: torch.Size([768, 768])
Original attention biases shape: torch.Size([768])


In [6]:
print("Original attention weights :", attention_weights)
print("Original attention biases :", attention_biases)

Original attention weights : Parameter containing:
tensor([[-0.0253,  0.0489, -0.0148,  ...,  0.0431, -0.0253,  0.0013],
        [-0.0143, -0.0465, -0.0020,  ..., -0.0073, -0.0296,  0.0208],
        [ 0.0147, -0.0400, -0.0344,  ..., -0.0697, -0.1240, -0.0031],
        ...,
        [-0.0214, -0.0160, -0.0680,  ...,  0.0233, -0.0359,  0.0355],
        [-0.0297,  0.0270,  0.0464,  ...,  0.0691,  0.0293,  0.0095],
        [-0.0012, -0.0403,  0.0343,  ..., -0.0742,  0.0371,  0.0421]],
       requires_grad=True)
Original attention biases : Parameter containing:
tensor([-1.7320e-01,  2.7564e-01,  1.6801e-01, -1.0946e-01,  1.6433e-01,
        -1.1501e-01, -2.5965e-01, -1.3676e-01, -3.0756e-01, -4.5222e-01,
         1.0455e-01, -8.2313e-02, -9.7356e-02, -2.0522e-02,  1.1782e-01,
         7.8542e-02, -1.1049e-01,  8.0185e-02,  3.4537e-02, -6.8996e-02,
        -6.8539e-04,  2.3121e-01,  1.2190e-01,  1.4254e-01, -7.6825e-02,
         1.5386e-01,  9.6756e-04,  1.0246e-01,  1.4043e-01, -1.5966e-01,


In [10]:
random_weights = torch.randn_like(attention_weights)
random_biases = torch.randn_like(attention_biases)


In [11]:
seventh_layer.attention.self.query.weight.data = random_weights
seventh_layer.attention.self.query.bias.data = random_biases

In [12]:
print("Modified attention weights:", seventh_layer.attention.self.query.weight)
print("Modified attention biases:", seventh_layer.attention.self.query.bias)

Modified attention weights: Parameter containing:
tensor([[ 1.1232, -2.0663, -0.3869,  ..., -0.9638,  1.0049, -0.1363],
        [ 0.3644, -0.4459,  0.3675,  ...,  0.5589,  0.2482,  0.7973],
        [ 0.5483,  1.7345, -0.1110,  ...,  0.7413, -0.2864,  0.5573],
        ...,
        [ 0.3874, -0.3777, -0.8853,  ...,  0.4760,  0.3183, -0.2340],
        [ 0.9746,  0.0902, -1.0719,  ..., -0.6945, -1.2466, -0.3071],
        [ 1.2965,  0.9028,  0.9281,  ..., -1.9690, -1.2167,  2.0736]],
       requires_grad=True)
Modified attention biases: Parameter containing:
tensor([-8.6173e-01, -7.4179e-01,  5.5969e-01, -3.4505e-01,  9.2275e-01,
        -9.2913e-01, -4.6782e-01,  3.9303e-01,  3.7233e-02,  9.5958e-01,
         1.7285e+00, -7.3191e-01,  5.0729e-01, -2.2224e+00,  5.2406e-01,
         5.7105e-01, -9.3041e-01, -1.1125e+00,  3.6708e-01, -3.5432e-01,
        -8.9508e-01, -5.0415e-02,  1.5396e+00, -8.0105e-01, -1.0225e+00,
        -1.5803e+00, -1.4517e+00, -8.0480e-01, -1.4903e+00, -3.7841e-01,
  